# Monthly Payment Features

In [1]:
import numpy as np
import pandas as pd
import gc

pd.set_option('display.float_format', '{:.10f}'.format)
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('./data/train.csv')
historical_transactions = pd.read_csv('./data/historical_transactions.csv')
new_merchant_transactions = pd.read_csv('./data/new_merchant_transactions.csv')

In [3]:
all_transactions = pd.concat([historical_transactions, new_merchant_transactions], axis=0)

In [4]:
all_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.7033309100,2017-06-25 15:33:07,1.0000000000,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.7331284800,2017-07-15 12:10:45,1.0000000000,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.7203860000,2017-08-09 22:04:29,1.0000000000,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.7353524100,2017-09-02 10:06:26,1.0000000000,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.7228653800,2017-03-10 01:14:19,1.0000000000,16,37


In [5]:
def shift_round(col):
    # shift_round: reverse normalization ?
    # where are the two numbers come from ?
    return np.round(col / 0.00150265118 + 497.06, 2)

all_transactions["purchase_amount_new"] = shift_round(all_transactions["purchase_amount"])

In [6]:
all_transactions.dtypes

authorized_flag          object
card_id                  object
city_id                   int64
category_1               object
installments              int64
category_3               object
merchant_category_id      int64
merchant_id              object
month_lag                 int64
purchase_amount         float64
purchase_date            object
category_2              float64
state_id                  int64
subsector_id              int64
purchase_amount_new     float64
dtype: object

In [7]:
all_transactions["month_abs"] = all_transactions.purchase_date.apply(lambda x: int(x[:7].replace("-", "")))

In [8]:
month_map = sorted(all_transactions.month_abs.unique())
month_map = dict(zip(month_map, range(len(month_map))))

In [9]:
all_transactions["month_lag"] = all_transactions.month_abs.map(month_map)

In [11]:
all_transactions.month_lag.value_counts().sort_index()

0     1147922
1     1206048
2     1456591
3     1448973
4     1562029
5     1698931
6     2045658
7     2233312
8     2432105
9     2547286
10    2872400
11    3480297
12    2937362
13    2541919
14     806902
15     657657
Name: month_lag, dtype: int64

In [19]:
all_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount_new,month_abs
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,5,-0.7033309100,2017-06-25 15:33:07,1.0000000000,16,37,29.0000000000,201706
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,6,-0.7331284800,2017-07-15 12:10:45,1.0000000000,16,16,9.1700000000,201707
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,7,-0.7203860000,2017-08-09 22:04:29,1.0000000000,16,37,17.6500000000,201708
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,8,-0.7353524100,2017-09-02 10:06:26,1.0000000000,16,34,7.6900000000,201709
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,2,-0.7228653800,2017-03-10 01:14:19,1.0000000000,16,37,16.0000000000,201703


# monthly_payment

In [23]:
monthly_payment = (all_transactions.groupby(["card_id", "month_lag"])
                                   .purchase_amount_new.sum()
                                   .reset_index(name="monthly_pa")
                                   .sort_values(["card_id", "month_lag"]))

In [24]:
monthly_payment.head()

,card_id,month_lag,monthly_pa
0,C_ID_00007093c1,1,1100.2000000000
1,C_ID_00007093c1,2,1168.9000000000
2,C_ID_00007093c1,3,1633.2500000000
3,C_ID_00007093c1,4,2691.1300000000
4,C_ID_00007093c1,5,2876.6300000000


# psum with rolling ratio

In [9]:
part = monthly_payment[:10000].copy()

part["ratio_1"] = part.rolling(window=2).monthly_pa.apply(lambda x: x[1] / x[0], raw=True)
part["lag_offset"] = part.rolling(window=2).month_lag.apply(lambda x: x[-1] - x[0], raw=True)
part.loc[part.lag_offset < 0, "ratio_1"] = np.NaN
del part["lag_offset"]

for i in range(3):
    window_size = i + 2
    sum_col = "sum_{}".format(window_size)
    ratio_col = "ratio_{}".format(window_size)
    part["lag_offset"] = part.rolling(window=window_size).month_lag.apply(lambda x: x[-1] - x[0], raw=True)
    part[sum_col] = part.rolling(window=window_size).monthly_pa.sum()
    part.loc[part.lag_offset < 0, sum_col] = np.NaN
    part[ratio_col] = part.rolling(window=2)[sum_col].apply(lambda x: x[1] / x[0], raw=True)
    part.loc[part.lag_offset < 0, ratio_col] = np.NaN
    del part["lag_offset"]

part["sum_1"] = part.monthly_pa

In [10]:
monthly_payment_features = pd.DataFrame()

for month in range(16):
    temp = part[part.month_lag == month_lag].set_index("card_id").drop(["month_lag", "monthly_pa"], axis=1).rename(columns=lambda x: "{}_lag{}".format(x, month_lag))
    monthly_payment_features = pd.concat([monthly_payment_features, temp], axis=1)

C:\Users\admin\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [25]:
def calc_monthly_payment_features(part):
    part["ratio_1"] = part.rolling(window=2).monthly_pa.apply(lambda x: x[1] / x[0], raw=True)
    part["lag_offset"] = part.rolling(window=2).month_lag.apply(lambda x: x[-1] - x[0], raw=True)
    part.loc[part.lag_offset < 0, "ratio_1"] = np.NaN
    del part["lag_offset"]

    for i in range(3):
        window_size = i + 2
        sum_col = "sum_{}".format(window_size)
        ratio_col = "ratio_{}".format(window_size)
        part["lag_offset"] = part.rolling(window=window_size).month_lag.apply(lambda x: x[-1] - x[0], raw=True)
        part[sum_col] = part.rolling(window=window_size).monthly_pa.sum()
        part.loc[part.lag_offset < 0, sum_col] = np.NaN
        part[ratio_col] = part.rolling(window=2)[sum_col].apply(lambda x: x[1] / x[0], raw=True)
        part.loc[part.lag_offset < 0, ratio_col] = np.NaN
        del part["lag_offset"]

    part["sum_1"] = part.monthly_pa
    
    monthly_payment_features = pd.DataFrame()

    for month in range(16):
        month_lag = month
        temp = part[part.month_lag == month_lag].set_index("card_id").drop(["month_lag", "monthly_pa"], axis=1).rename(columns=lambda x: "{}_lag{}".format(x, month_lag))
        monthly_payment_features = pd.concat([monthly_payment_features, temp], axis=1)
    return monthly_payment_features

In [26]:
from data_io import features_downcast
from downcast import save_dataframe32, load_dataframe32

In [27]:
features_df = calc_monthly_payment_features(monthly_payment)
features_df.shape
features_df.index.nunique()
features_df.reset_index().head()

save_dataframe32(features_downcast("monthly_psum_abs"),
                 features_df.reset_index().rename(columns={"index":"card_id"}), keep=["card_id"])

# next

1. pmax with rolling ratio
2. c-m std
3. m std
4. max-m count
5. max-m count/ month count
6. max-m std

# pmax with rolling ratio

In [30]:
monthly_payment_pmax = (all_transactions.groupby(["card_id", "month_lag"])
                                        .purchase_amount_new.max()
                                        .reset_index(name="monthly_pa")
                                        .sort_values(["card_id", "month_lag"]))

In [31]:
monthly_payment_pmax.head()

,card_id,month_lag,monthly_pa
0,C_ID_00007093c1,1,521.8000000000
1,C_ID_00007093c1,2,230.0000000000
2,C_ID_00007093c1,3,500.0000000000
3,C_ID_00007093c1,4,702.0000000000
4,C_ID_00007093c1,5,500.0000000000


In [32]:
features_df = calc_monthly_payment_features(monthly_payment_pmax)
save_dataframe32(features_downcast("monthly_pmax_abs"),
                 features_df.reset_index().rename(columns={"index":"card_id"}), keep=["card_id"])

# main_merchant

In [45]:
all_transactions.card_id.nunique()

325540

In [12]:
monthly_payment_merchant_psum =  (all_transactions.groupby(["card_id", "month_lag", "merchant_id"])
                                                  .purchase_amount_new.sum()
                                                  .reset_index(name="monthly_pa")
                                                  .sort_values(["card_id", "month_lag"]))

In [13]:
monthly_payment_merchant_psum.head()

,card_id,month_lag,merchant_id,monthly_pa
0,C_ID_00007093c1,1,M_ID_69423b34e4,521.8000000000
1,C_ID_00007093c1,1,M_ID_9400cf2342,116.0000000000
2,C_ID_00007093c1,1,M_ID_a33355a1b7,462.4000000000
3,C_ID_00007093c1,2,M_ID_69423b34e4,515.9000000000
4,C_ID_00007093c1,2,M_ID_9400cf2342,653.0000000000


In [14]:
monthly_payment_merchant_psum_pmax = (monthly_payment_merchant_psum.groupby(["card_id", "month_lag"])
                                                                   .monthly_pa.max()
                                                                   .reset_index(name="monthly_pa")
                                                                   .sort_values(["card_id", "month_lag"]))

In [15]:
monthly_payment_merchant_psum_pmax.head()

,card_id,month_lag,monthly_pa
0,C_ID_00007093c1,1,521.8000000000
1,C_ID_00007093c1,2,653.0000000000
2,C_ID_00007093c1,3,718.4500000000
3,C_ID_00007093c1,4,2204.0000000000
4,C_ID_00007093c1,5,1400.0000000000


In [47]:
monthly_payment_merchant_psum["card_id_extend"] = monthly_payment_merchant_psum.card_id + "_" + monthly_payment_merchant_psum.month_lag.astype(str)
monthly_payment_merchant_psum_pmax["card_id_extend"] = monthly_payment_merchant_psum_pmax.card_id + "_" + monthly_payment_merchant_psum_pmax.month_lag.astype(str)
monthly_payment_merchant_psum["monthly_pa_max"] = monthly_payment_merchant_psum.card_id_extend.map(monthly_payment_merchant_psum_pmax.set_index("card_id_extend").monthly_pa)
monthly_main_merchant = monthly_payment_merchant_psum[monthly_payment_merchant_psum.monthly_pa == monthly_payment_merchant_psum.monthly_pa_max]

In [66]:
monthly_main_merchant.head()

,card_id,month_lag,merchant_id,monthly_pa,card_id_extend,monthly_pa_max,m_code
0,C_ID_00007093c1,1,M_ID_69423b34e4,521.8000000000,C_ID_00007093c1_1,521.8000000000,100434
4,C_ID_00007093c1,2,M_ID_9400cf2342,653.0000000000,C_ID_00007093c1_2,653.0000000000,141523
6,C_ID_00007093c1,3,M_ID_69423b34e4,718.4500000000,C_ID_00007093c1_3,718.4500000000,100434
11,C_ID_00007093c1,4,M_ID_15446e939c,2204.0000000000,C_ID_00007093c1_4,2204.0000000000,20468
22,C_ID_00007093c1,5,M_ID_ee8fcd02ca,1400.0000000000,C_ID_00007093c1_5,1400.0000000000,227801


In [49]:
from sklearn.preprocessing import LabelEncoder
m_ecd = LabelEncoder()
monthly_main_merchant["m_code"] = m_ecd.fit_transform(monthly_main_merchant.merchant_id)

C:\Users\admin\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [50]:
monthly_main_merchant.shape

(3053315, 7)

In [51]:
monthly_main_merchant.head()

,card_id,month_lag,merchant_id,monthly_pa,card_id_extend,monthly_pa_max,m_code
0,C_ID_00007093c1,1,M_ID_69423b34e4,521.8000000000,C_ID_00007093c1_1,521.8000000000,100434
4,C_ID_00007093c1,2,M_ID_9400cf2342,653.0000000000,C_ID_00007093c1_2,653.0000000000,141523
6,C_ID_00007093c1,3,M_ID_69423b34e4,718.4500000000,C_ID_00007093c1_3,718.4500000000,100434
11,C_ID_00007093c1,4,M_ID_15446e939c,2204.0000000000,C_ID_00007093c1_4,2204.0000000000,20468
22,C_ID_00007093c1,5,M_ID_ee8fcd02ca,1400.0000000000,C_ID_00007093c1_5,1400.0000000000,227801


In [52]:
monthly_main_merchant = monthly_main_merchant.drop_duplicates(subset="card_id_extend")
monthly_main_merchant.shape

(3040375, 7)

In [57]:
monthly_payment_features = pd.DataFrame({
    "main_m_lag{}".format(month): monthly_main_merchant[monthly_main_merchant.month_lag == month].set_index("card_id").m_code + 1
    for month in range(16)
})

In [64]:
monthly_main_merchant.merchant_id.nunique()

244022

In [61]:
monthly_payment_features = monthly_payment_features.fillna(0).astype(int)

In [58]:
monthly_payment_features.shape

(325540, 16)

In [67]:
monthly_payment_pa = pd.DataFrame({
    "main_m_lag{}".format(month): monthly_main_merchant[monthly_main_merchant.month_lag == month].set_index("card_id").monthly_pa
    for month in range(16)
})

In [68]:
monthly_payment_pa.head()

,main_m_lag0,main_m_lag1,main_m_lag2,main_m_lag3,main_m_lag4,main_m_lag5,main_m_lag6,main_m_lag7,main_m_lag8,main_m_lag9,main_m_lag10,main_m_lag11,main_m_lag12,main_m_lag13,main_m_lag14,main_m_lag15
C_ID_00007093c1,nan,521.8000000000,653.0000000000,718.4500000000,2204.0000000000,1400.0000000000,331.0000000000,2900.0000000000,220.0000000000,413.0000000000,527.0000000000,782.0000000000,430.0000000000,500.0000000000,nan,60.0000000000
C_ID_0001238066,nan,nan,nan,nan,nan,nan,nan,nan,127.1300000000,430.0100000000,661.5600000000,792.7100000000,1008.2200000000,771.8300000000,444.9400000000,250.0000000000
C_ID_0001506ef0,39.0000000000,79.0000000000,58.2000000000,nan,30.0000000000,16.3300000000,1017.0000000000,842.0000000000,750.0000000000,910.7400000000,881.1400000000,750.0000000000,1100.0000000000,1491.0000000000,9.9200000000,nan
C_ID_0001793786,1174.8300000000,1475.5800000000,2290.1800000000,2078.1700000000,1621.6100000000,3527.8000000000,5733.9000000000,1650.8000000000,3445.7600000000,1443.2500000000,2580.0000000000,1500.0000000000,nan,nan,nan,nan
C_ID_000183fdda,nan,nan,nan,nan,nan,nan,nan,2158.2000000000,609.2000000000,541.2200000000,1764.9000000000,1960.0000000000,2337.0000000000,1374.9300000000,191.2000000000,425.4000000000


In [63]:
monthly_payment_features.to_csv("./data/train_test_main_merchant.csv")

In [69]:
monthly_payment_pa.to_csv("./data/train_test_main_merchant_pa.csv")

# merchant_length

more
1. daynum
2. ratio

In [38]:
merchant_dura = (monthly_payment_merchant_psum.groupby(["card_id", "merchant_id"])
                                              .size()
                                              .reset_index(name="merchant_dura"))

In [40]:
merchant_dura.head()

,card_id,merchant_id,merchant_dura
0,C_ID_00007093c1,M_ID_00a6ca8a8a,1
1,C_ID_00007093c1,M_ID_0379adb435,4
2,C_ID_00007093c1,M_ID_06a8d84366,1
3,C_ID_00007093c1,M_ID_08f01305af,1
4,C_ID_00007093c1,M_ID_0dabc5a70c,2


In [41]:
merchant_dura_count = (merchant_dura.groupby(["card_id", "merchant_dura"])
                                    .size()
                                    .reset_index(name="count"))

In [42]:
merchant_dura_count.head()

,card_id,merchant_dura,count
0,C_ID_00007093c1,1,15
1,C_ID_00007093c1,2,7
2,C_ID_00007093c1,3,5
3,C_ID_00007093c1,4,1
4,C_ID_00007093c1,7,1


In [45]:
df_features = pd.DataFrame()

for dura in range(14):
    temp = (merchant_dura_count[merchant_dura_count.merchant_dura == dura + 1].set_index("card_id")
                                                                              .drop(["merchant_dura"], axis=1)
                                                                              .rename(columns=lambda x: "{}_dura{}".format(x, dura + 1)))
    df_features = pd.concat([df_features, temp], axis=1)

In [49]:
df_features["count_total"] = df_features.sum(axis=1)

for dura in range(14):
    count_col = "count_dura{}".format(dura + 1)
    ratio_col = "ratio_dura{}".format(dura + 1)
    df_features[ratio_col] = df_features[count_col] / df_features["count_total"]

In [51]:
save_dataframe32(features_downcast("duar_count"),
                 df_features.reset_index().rename(columns={"index":"card_id"}), keep=["card_id"])

In [50]:
df_features.head().T

,C_ID_00007093c1,C_ID_0001238066,C_ID_0001506ef0,C_ID_0001793786,C_ID_000183fdda
count_dura1,15.0000000000,82.0000000000,22.0000000000,128.0000000000,62.0000000000
count_dura2,7.0000000000,4.0000000000,3.0000000000,12.0000000000,12.0000000000
count_dura3,5.0000000000,1.0000000000,nan,3.0000000000,6.0000000000
count_dura4,1.0000000000,nan,2.0000000000,2.0000000000,3.0000000000
count_dura5,nan,2.0000000000,nan,2.0000000000,1.0000000000
count_dura6,nan,1.0000000000,1.0000000000,1.0000000000,nan
count_dura7,1.0000000000,nan,nan,2.0000000000,nan
count_dura8,nan,nan,nan,nan,nan
count_dura9,nan,nan,1.0000000000,nan,nan
count_dura10,1.0000000000,nan,nan,nan,nan


In [39]:
merchant_dura.merchant_dura.value_counts()

1     9687983
2     1605279
3      671822
4      350670
5      204864
6      129588
7       84699
8       57202
9       40143
10      28105
11      20239
12      15009
13      11280
14      10637
Name: merchant_dura, dtype: int64

# merchant avg std

In [52]:
merchant_avg_std = (all_transactions.groupby(["merchant_id", "card_id"])
                                    .purchase_amount_new.std().reset_index(name="pa_std")
                                    .groupby("merchant_id").pa_std.mean().reset_index(name="pa_std_avg"))

In [53]:
merchant_avg_std.head()

,merchant_id,pa_std_avg
0,M_ID_000025127f,0.5773502692
1,M_ID_0000699140,27.1220987316
2,M_ID_00006a5552,nan
3,M_ID_000087311e,269.2202923001
4,M_ID_0000ab0b2d,5.0041993186


In [54]:
save_dataframe32(features_downcast("merchant_avg_std"),
                 merchant_avg_std, keep=["merchant_id"])

In [57]:
monthly_payment_merchant_psum.head()

,card_id,month_lag,merchant_id,monthly_pa
0,C_ID_00007093c1,1,M_ID_69423b34e4,521.8000000000
1,C_ID_00007093c1,1,M_ID_9400cf2342,116.0000000000
2,C_ID_00007093c1,1,M_ID_a33355a1b7,462.4000000000
3,C_ID_00007093c1,2,M_ID_69423b34e4,515.9000000000
4,C_ID_00007093c1,2,M_ID_9400cf2342,653.0000000000


In [58]:
monthly_payment_merchant_psum["card_id_extend"] = monthly_payment_merchant_psum.card_id + "_" + monthly_payment_merchant_psum.month_lag.astype(str)
monthly_payment_merchant_psum_pmax["card_id_extend"] = monthly_payment_merchant_psum_pmax.card_id + "_" + monthly_payment_merchant_psum_pmax.month_lag.astype(str)
monthly_payment_merchant_psum["monthly_pa_max"] = monthly_payment_merchant_psum.card_id_extend.map(monthly_payment_merchant_psum_pmax.set_index("card_id_extend").monthly_pa)
monthly_main_merchant = monthly_payment_merchant_psum[monthly_payment_merchant_psum.monthly_pa == monthly_payment_merchant_psum.monthly_pa_max]

In [59]:
monthly_main_merchant["monthly_pa"] = monthly_main_merchant.merchant_id.map(merchant_avg_std.set_index("merchant_id").pa_std_avg)

In [62]:
monthly_main_merchant = monthly_main_merchant.drop_duplicates("card_id_extend")

In [63]:
monthly_main_merchant.head()

,card_id,month_lag,merchant_id,monthly_pa,card_id_extend,monthly_pa_max
0,C_ID_00007093c1,1,M_ID_69423b34e4,44.2637050280,C_ID_00007093c1_1,521.8000000000
4,C_ID_00007093c1,2,M_ID_9400cf2342,68.3237688669,C_ID_00007093c1_2,653.0000000000
6,C_ID_00007093c1,3,M_ID_69423b34e4,44.2637050280,C_ID_00007093c1_3,718.4500000000
11,C_ID_00007093c1,4,M_ID_15446e939c,29.2447842358,C_ID_00007093c1_4,2204.0000000000
22,C_ID_00007093c1,5,M_ID_ee8fcd02ca,137.8139649003,C_ID_00007093c1_5,1400.0000000000


In [64]:
features_df = calc_monthly_payment_features(monthly_main_merchant[["card_id", "month_lag", "monthly_pa"]])
save_dataframe32(features_downcast("monthly_merchant_avg_std_abs"),
                 features_df.reset_index().rename(columns={"index":"card_id"}), keep=["card_id"])

In [65]:
features_df.head().T

,C_ID_00007093c1,C_ID_0001238066,C_ID_0001506ef0,C_ID_0001793786,C_ID_000183fdda
ratio_1_lag0,nan,nan,nan,nan,nan
sum_2_lag0,nan,nan,nan,nan,nan
ratio_2_lag0,nan,nan,nan,nan,nan
sum_3_lag0,nan,nan,nan,nan,nan
ratio_3_lag0,nan,nan,nan,nan,nan
sum_4_lag0,nan,nan,nan,nan,nan
ratio_4_lag0,nan,nan,nan,nan,nan
sum_1_lag0,nan,nan,7.1392607660,83.1361047020,nan
ratio_1_lag1,nan,nan,2.1933054913,nan,nan
sum_2_lag1,nan,nan,22.7978406080,nan,nan
